# 7506 - Trabajo Práctico 2

---

## Introducción

### Librerías

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Dataset Train preprocesado
train_ds = pd.read_csv('datasets/tp1-train.csv')
train_ds = train_ds.drop(['Unnamed: 0'], axis=1)
train_ds.head()

In [ ]:
# Dataset Test preprocesado
test_ds = pd.read_csv('datasets/tp1-test.csv')
test_ds = test_ds.drop(['Unnamed: 0'], axis=1)
test_ds

## Procesamiento del lenguaje natural

### Ampliación del dataset

### Modelos

## Redes Neuronales

### Regresión

Predecir el precio de la propiedad y utilizar como métrica de evaluación el error cuadrático medio.

Vamos a predecir el precio de la propiedad (dolares) en base a la superifice total y cubierta. Ya que como vimos en el trabajo pasado, eran los atributos que mas se correlacionaban con el precio.

In [ ]:
ds_trabajo = train_ds.drop(['start_date', 'end_date', 'place_l3', 'property_title'], axis=1)
ds_trabajo.head()

In [ ]:
ds_trabajo.shape

In [ ]:
columnas_predictoras = ds_trabajo.columns.to_list()
columnas_predictoras.remove('property_price')
columnas_predictoras

In [ ]:
variable_predictora = "property_surface_total"

x_train, x_test, y_train, y_test = train_test_split(ds_trabajo.loc[:,variable_predictora], ds_trabajo.loc[:,variable_predictora], test_size=0.2)

In [ ]:
x_train

In [ ]:
sscaler = StandardScaler()
sscaler.fit(pd.DataFrame(x_train))

In [ ]:
x_train_transform = sscaler.transform(pd.DataFrame(x_train))
x_test_transform = sscaler.transform(pd.DataFrame(x_test))

#### Modelo 1

In [ ]:
# Creo un modelo Red Neuronal
d_in=1
d_out=1

modelo1 = keras.Sequential([
    keras.layers.Dense(1,input_shape=(d_in,))]) #, activation="relu"

In [ ]:
modelo1.summary()

In [ ]:
plot_regression(modelo1, x_train_transform, y_train,title = f"Modelo inicial", xlabel="property_surface_total", ylabel="property_price")

In [ ]:
modelo1.compile(
  optimizer=keras.optimizers.SGD(learning_rate=0.001), 
  loss='mse', 
  metrics=['mae'], 
)

In [ ]:
# Entrenamiento del modelo
modelo1.fit(x_train_transform,y_train,epochs=100,batch_size=16,verbose=False)

Miremos como la red aprendio los datos con los que fue entrenada

In [ ]:
plot_regression(modelo1, x_train_transform, y_train, title = f"Modelo Final",xlabel = "property_surface_total", ylabel="property_price")

Miremos como la red predice

In [ ]:
plot_regression(modelo1, x_test_transform, y_test,title = f"Modelo Final", xlabel="property_surface_total", ylabel="property_price")

In [ ]:
y_pred_modelo1 = modelo1.predict(x_test_transform)

mae_modelo1 = mean_absolute_error(y_test,y_pred_modelo1)
mse_modelo1 = mean_squared_error(y_test,y_pred_modelo1)

print(f"Error absoluto medio {mae_modelo1}")
print(f"Error cuadrático medio {mse_modelo1}")

### Clasificación

#### Construcción del target

#### Modelo

## Ensambles de modelos

### Ensamble 1

### Ensamble 2

## Conclusiones